In [18]:
import torch
import numpy as np
import time
from tqdm import tqdm

In [19]:
from Models.MLP import SimpleMLP, train_step, val_step


In [30]:
targcols

array(['ccn_001', 'ccn_003', 'ccn_006', 'CHI', 'CHI_CCN', 'D_ALPHA',
       'D_GAMMA', 'D_ALPHA_CCN', 'D_GAMMA_CCN', 'PM25'], dtype='<U11')

In [23]:
ckpt_name = "ep_13_sunny_D_ALPHA,D_GAMMA,D_ALPHA_CCN,D_GAMMA_CCN_XLONG,XLAT_MLP.pt"
ckpt = torch.load(ckpt_name)

In [24]:
def getting_datapaths(
    train_idx,
    val_idx,
    files='sunny', 
    target_cols=["ccn_001", "ccn_003", "ccn_006"],
    exclude_cols = ["XLONG", "XLAT"],
):
    # getting colname
    featloc = "/home/kphadke/hackathon/norm_data_timesteps_feat"
    targloc = "/home/kphadke/hackathon/norm_data_timesteps_targ"

    featcols = np.load(f"{featloc}/x_name.npy")
    targcols = np.load(f"{targloc}/y_name.npy")
    
    target_idx = [np.where(targcols == i)[0].item() for i in target_cols ]
    feat_idx   = [np.where(featcols == i)[0].item() for i in featcols if i not in exclude_cols]
    
    # getting paths
    if files == 'sunny':
        # my data
        featloc = "/home/kwoksun2/hackathon_data/norm_data"
        targloc = "/home/kwoksun2/hackathon_data/norm_data"

        train_feat_files = [f"{featloc}/t{t}_feat.npy" for t in train_idx]
        val_feat_files   = [f"{featloc}/t{t}_feat.npy" for t in val_idx  ]

        train_targ_files = [f"{targloc}/t{t}_targ.npy" for t in train_idx]
        val_targ_files   = [f"{targloc}/t{t}_targ.npy" for t in val_idx]
    
    elif files == 'kedar':
        featloc = "/home/kphadke/hackathon/norm_data_timesteps_feat"
        targloc = "/home/kphadke/hackathon/norm_data_timesteps_targ"

        train_feat_files = [f"{featloc}/{t}feat_norm.npy" for t in train_idx]
        val_feat_files   = [f"{featloc}/{t}feat_norm.npy" for t in val_idx  ]

        train_targ_files = [f"{targloc}/{t}targ_norm.npy" for t in train_idx]
        val_targ_files   = [f"{targloc}/{t}targ_norm.npy" for t in val_idx]

    return [np.array(train_feat_files),np.array(train_targ_files)], [np.array(val_feat_files),np.array(val_targ_files)], feat_idx, target_idx



model = ckpt['model_state_dict']
datasource = ckpt['datasource']
exclude_cols =  ckpt['exclude_cols']
target_cols =  ckpt['target_cols']

# getting the idx used in the model
featloc = "/home/kphadke/hackathon/norm_data_timesteps_feat"
targloc = "/home/kphadke/hackathon/norm_data_timesteps_targ"

featcols = np.load(f"{featloc}/x_name.npy")
targcols = np.load(f"{targloc}/y_name.npy")

target_idx = [np.where(targcols == i)[0].item() for i in target_cols ]
feat_idx   = [np.where(featcols == i)[0].item() for i in featcols if i not in exclude_cols]

from sklearn.model_selection import train_test_split

# THIS IS JUST FOR TESTING!!!!!!!
# WILL CHANGE BACK TO 133 later
nsnapshots = 133

# DONT CHANGE THE SEEEDDDDDD!!!!!
np.random.seed(52)
full_indx = np.arange(nsnapshots)
train_idx, val_idx = train_test_split(full_indx, test_size=0.2, train_size=0.8) 

train_files, val_files, feat_idx, target_idx = getting_datapaths(
    train_idx, 
    val_idx, 
    files=datasource, 
    exclude_cols = exclude_cols,
    target_cols  = target_cols
)

In [25]:
# kedar
if datasource == 'kedar':
    test_feats = [f"/home/kphadke/hackathon/norm_data_timesteps_feat_eval/{i}feat_norm_eval.npy" for i in range(12)]
    test_targs = [f"/home/kphadke/hackathon/norm_data_timesteps_targ_eval/{i}targ_norm_eval.npy" for i in range(12)]
else:
    test_feats = [f"/home/kwoksun2/hackathon_data/norm_data_val/t{i}_feat.npy" for i in range(12)]
    test_targs = [f"/home/kwoksun2/hackathon_data/norm_data_val/t{i}_targ.npy" for i in range(12)]
#     raise ValueError
    
if datasource == "sunny":
    target_std= np.array(
        [1.3428156,
        1.0779657,
        1.0479535,
        0.08097392,
        0.10943509 ,
        0.20763765 ,
        0.2632665 ,
        0.07147439,
        0.091097355,
        1.1029556
        ])
    
    tstd = target_std[target_idx]
    
elif datasource == "kedar":
    target_std = np.load("/home/kphadke/hackathon/targ_std.npy").transpose()
    tstd = target_std[:,target_idx]
    tstd = tstd[:,np.newaxis,np.newaxis,:]

In [26]:
test_files = [test_feats, test_targs]

In [27]:
targcols

array(['ccn_001', 'ccn_003', 'ccn_006', 'CHI', 'CHI_CCN', 'D_ALPHA',
       'D_GAMMA', 'D_ALPHA_CCN', 'D_GAMMA_CCN', 'PM25'], dtype='<U11')

In [28]:
import os

folder_name = f'predictions_{ckpt_name.replace(".pt","")}'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)

In [32]:
# create a prediction dump
feat_files, targ_files = test_files
# we are selecting only the first validation data
for i in range(len(feat_files)):
    feat_data = np.load(feat_files[i])
    targ_data = np.load(targ_files[i])

    if targ_data.ndim == 1:
        targ_data = targ_data.reshape(10,-1).transpose()

    tds = torch.utils.data.TensorDataset(
        torch.from_numpy(feat_data[:,feat_idx]).float(), 
        torch.from_numpy(targ_data[:,target_idx]).float()
    )
    dl = torch.utils.data.DataLoader(tds, batch_size=1024, shuffle=False)

    # turn it in to a data loader for accessing
    ypred = []
    ytruth = []
    xfeats = []
    with torch.no_grad():
        for feat, targ in dl:
            ypred.append(model(feat.cuda()).cpu().numpy()[:,:4])
            ytruth.append(targ.numpy()[:,:4])
            xfeats.append(feat.numpy())

    ypred = np.vstack(ypred)
    ytruth = np.vstack(ytruth)
    xfeats = np.vstack(xfeats)
    
    np.save(f"{folder_name}/{i}_xfeats.npy", xfeats)
    np.save(f"{folder_name}/{i}_ypred.npy", ypred)
    np.save(f"{folder_name}/{i}_ytruth.npy", ytruth)